# 1. Necessary Headfiles

In [8]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from PIL import Image

# 2. Prepare Datasets

## 2.1.1 (Original Data) Produce Datamap
Navigate to the data folder 

In [5]:
root_dir = "../eegdata"
# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "rest"
# condition_dir: hc, mcs, uws
condi_dir = "hc"
# distribution index matrix
matrix_index = np.array([
    [0,0,2,2,4,4,6,6,8,8,2,2,4,4,6,6,2,4,6,3,3,5,5,3,3,5,5,3,3,2,2,4,4,6,6,1,1,3,3,5,5,7,7,2,2,4,4,6,6,1,1,3,3,5,5,7,7,0,5],
    [4,6,3,7,3,7,3,7,4,6,1,9,1,9,1,9,5,5,5,4,6,4,6,2,8,2,8,0,10,4,6,4,6,4,6,4,6,3,7,3,7,4,6,2,8,2,8,2,8,3,7,1,9,1,9,3,7,5,5]
])

### Conditional Data

In [ ]:
window_overlap = 0
# 设置使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = MyData(root_dir, exper_dir, condi_dir) # eg.conditionA\hc

for person in tqdm(range(len(dataset))): #12种情况中的1种，其中的所有被试
    # filepath是最子文件夹中每个.mat文件的名字
    # path是包含当前情况的.mat文件的子文件夹
    filename = os.path.join(dataset.path, dataset.file_path[person]) # eg. conditionA\hc\hc1.set.mat
    data = scipy.io.loadmat(filename) # 读取该被试的数据为字典
    data = data['datas'] # 键值对读取59*x的数据矩阵
    # 将数据移动到GPU上
    data = torch.from_numpy(data).to(device)
    
    # 在GPU上处理数据
    # 创建被试全0map
    data_map_person = []
    print(f"----现在处理：{dataset.file_path[person]}，共{data.shape[2]}段----")
    # 遍历该被试的所有段
    for para in tqdm(range(data.shape[2])): 
        # 创建段全0map
        data_map_para = [] 
        for point in range(data.shape[1]): # 遍历该段所有数据点
            # 创建单个数据点全0map
            data_map_point = torch.zeros((10, 11), device=device) 
            # 遍历59电极数据并赋值给全0map
            for channel in range(data.shape[0]): 
                data_map_point[matrix_index[0][channel]][matrix_index[1][channel]] = data[channel][point][para]
            # 保存单个数据点map到段map列表
            data_map_para.append(data_map_point)
        # 保存 段map 到 被试map
        data_map_person.append(torch.stack(data_map_para))
        print(len(data_map_person))
        # 清理内存
        del data_map_para
        gc.collect()
        torch.cuda.empty_cache()
    # 保存 被试map 到文件中
    save_path = f"../eegmap_direct/conditionC/uws/{dataset.file_path[person]}.pt"
    torch.save(torch.stack(data_map_person), save_path)

### rest data

In [11]:
root_dir = "../eegdata"
# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "rest"
# condition_dir: hc, mcs, uws
condi_dir = "hc"
# distribution index matrix
matrix_index = np.array([
    [0,0,2,2,4,4,6,6,8,8,2,2,4,4,6,6,2,4,6,3,3,5,5,3,3,5,5,3,3,2,2,4,4,6,6,1,1,3,3,5,5,7,7,2,2,4,4,6,6,1,1,3,3,5,5,7,7,0,5],
    [4,6,3,7,3,7,3,7,4,6,1,9,1,9,1,9,5,5,5,4,6,4,6,2,8,2,8,0,10,4,6,4,6,4,6,4,6,3,7,3,7,4,6,2,8,2,8,2,8,3,7,1,9,1,9,3,7,5,5]
])
# 设置使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = MyData(root_dir, exper_dir, condi_dir) # eg.conditionA\hc

for person in tqdm(range(len(dataset))): #12种情况中的1种，其中的所有被试
    # filepath是最子文件夹中每个.mat文件的名字
    # path是包含当前情况的.mat文件的子文件夹
    filename = os.path.join(dataset.path, dataset.file_path[person]) # eg. conditionA\hc\hc1.set.mat
    data = scipy.io.loadmat(filename) # 读取该被试的数据为字典
    data = data['datas'] # 键值对读取59*x的数据矩阵
    # 将数据移动到GPU上
    data = torch.from_numpy(data).to(device)
    
    # 在GPU上处理数据
    # 创建被试全0map
    data_map_person = []
    print(f"----现在处理：{dataset.file_path[person]}，共{data.shape[1]}个数据点----")
    # 遍历该被试的所有段
    for point in tqdm(range(data.shape[1])): # 遍历该段所有数据点
        # 创建单个数据点全0map
        data_map_point = torch.zeros((10, 11), device=device) 
        # 遍历59电极数据并赋值给全0map
        for channel in range(data.shape[0]): 
            data_map_point[matrix_index[0][channel]][matrix_index[1][channel]] = data[channel][point]
        # 保存单个数据点map到被试map列表
        data_map_person.append(data_map_point)
    # 清理内存
    gc.collect()
    torch.cuda.empty_cache()
    # 保存 被试map 到文件中
    save_path = f"../eegmap_direct/rest/hc/{dataset.file_path[person]}.pt"
    torch.save(torch.stack(data_map_person), save_path)

----现在处理：hc1.set.mat，共116241个数据点----


----现在处理：hc10.set.mat，共151738个数据点----


----现在处理：hc11.set.mat，共97794个数据点----


----现在处理：hc12.set.mat，共110221个数据点----


----现在处理：hc13.set.mat，共96004个数据点----


...

----现在处理：hc5.set.mat，共118235个数据点----


----现在处理：hc6.set.mat，共113920个数据点----


----现在处理：hc7.set.mat，共113935个数据点----


----现在处理：hc8.set.mat，共125408个数据点----


----现在处理：hc9.set.mat，共118521个数据点----


## 2.1.2 Split data into 2400-points pieces

### Conditional Data

In [31]:
root_dir = "../data/eegmap_direct"
exper_dir = "conditionB"
condi_dir = "hc"

dataset = MyData(root_dir, exper_dir, condi_dir)
data_all = []
for person in tqdm(range(len(dataset))):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(filename)
    print(data_map.size())
    
    for i in tqdm(range(data_map.size(0))): # 每个被试切割成2400大小的块保存到data_all中
        data_all.append(data_map[i])
                        
savepath = f"../data/eegmap_chunks/{exper_dir}/{condi_dir}/{exper_dir}_{condi_dir}.pt"
torch.save(torch.stack(data_all),savepath)
print(len(data_all))
del data_all
del dataset
del data_map
gc.collect()
torch.cuda.empty_cache()

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/20 [00:00<?, ?it/s]

../data/eegmap_direct\conditionC\hc\hc1.set.mat.pt
torch.Size([62, 2400, 10, 11])


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/62 [00:00<?, ?it/s]

...

../data/eegmap_direct\conditionC\hc\hc8.set.mat.pt
torch.Size([61, 2400, 10, 11])


  0%|          | 0/61 [00:00<?, ?it/s]

../data/eegmap_direct\conditionC\hc\hc9.set.mat.pt
torch.Size([63, 2400, 10, 11])


  0%|          | 0/63 [00:00<?, ?it/s]

1231


### Rest Data

In [13]:
root_dir = "../data/eegmap_direct"
exper_dir = "rest"
condi_dir = "uws"

dataset = MyData(root_dir, exper_dir, condi_dir)
data_all = []
for person in tqdm(range(len(dataset))):
    filename = os.path.join(dataset.path, dataset.file_path[person])
    data_map = torch.load(filename)
    print(filename)
    print(data_map.size())
    
    # 计算可以切割的最大块数
    num_chunks = data_map.size(0) // 2400
    # 裁剪数据，使其可以被 2400 整除
    cropped_data_map = data_map[:num_chunks * 2400]
    # 切割成块
    data_map_splitted = [chunk for chunk in torch.chunk(cropped_data_map, chunks=num_chunks, dim=0)]
    data_map_splitted_tensor = torch.stack(data_map_splitted, dim=0)
    print(data_map_splitted_tensor.size())
    
    for i in tqdm(range(data_map_splitted_tensor.size(0))): # 每个被试切割成2400大小的块保存到data_all中
        data_map_chunk = data_map_splitted_tensor[i]
        data_all.append(data_map_chunk)
savepath = f"../data/eegmap_chunks/{exper_dir}/{condi_dir}/rest_{condi_dir}.pt"
torch.save(torch.stack(data_all),savepath)
print(len(data_all))
del dataset
del data_all
del data_map
del cropped_data_map
del data_map_splitted
del data_map_splitted_tensor
del data_map_chunk
gc.collect()
torch.cuda.empty_cache()

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/18 [00:00<?, ?it/s]

../data/eegmap_direct\rest\uws\uws1.set.mat.pt
torch.Size([122560, 10, 11])
torch.Size([51, 2400, 10, 11])


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/51 [00:00<?, ?it/s]

...

../data/eegmap_direct\rest\uws\uws8.set.mat.pt
torch.Size([118679, 10, 11])
torch.Size([49, 2400, 10, 11])


  0%|          | 0/49 [00:00<?, ?it/s]

../data/eegmap_direct\rest\uws\uws9.set.mat.pt
torch.Size([115715, 10, 11])
torch.Size([48, 2400, 10, 11])


  0%|          | 0/48 [00:00<?, ?it/s]

872


## 2.2.1 Slide Window(Abandoned)

In [7]:
# 定义文件夹路径
root_dir = "../data/eegmap_direct"
exper_dir = "rest"
condi_dir = "hc"
folder_path = os.path.join(root_dir, exper_dir, condi_dir)

# 定义滑动窗口参数
window_size = 2400
step_size = 200

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.pt'):
        # 拼接文件路径
        file_path = os.path.join(folder_path, filename)
        # 加载.pt文件
        eegmap = torch.load(file_path)
        # 显示当前处理文件
        print(f'Loaded file: {filename}')
        # 切割为滑动窗口
        num_windows = (eegmap.size(0) - window_size) // step_size + 1
        window_group = []
        for i in tqdm(range(num_windows)):
            start = i * step_size
            end = start + window_size
            window = eegmap[start:end]
            # 在这里对切割得到的窗口数据进行处理
#             print(window.size())
            name_parts = filename.split('.')
            condi_name = name_parts[0]
#             print(hc_name)
            # 将 window 存储到列表或其他数据结构中，或者进行进一步的操作
            window_group.append(window)
            del window
            gc.collect()
            torch.cuda.empty_cache()
        print(torch.stack(window_group).size())
        torch.save(torch.stack(window_group),f"../data/eegmap_overlap_200/{exper_dir}/{condi_dir}/{condi_name}_over200.pt")
        del eegmap
        del window_group
        gc.collect()
        torch.cuda.empty_cache()

Loaded file: hc1.set.mat.pt


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/570 [00:00<?, ?it/s]

torch.Size([570, 2400, 10, 11])
Loaded file: hc10.set.mat.pt


  0%|          | 0/747 [00:00<?, ?it/s]

...

torch.Size([558, 2400, 10, 11])
Loaded file: hc8.set.mat.pt


  0%|          | 0/616 [00:00<?, ?it/s]

torch.Size([616, 2400, 10, 11])
Loaded file: hc9.set.mat.pt


  0%|          | 0/581 [00:00<?, ?it/s]

torch.Size([581, 2400, 10, 11])
